# Support Vector Machines

[Reference 1](https://scikit-learn.org/stable/modules/svm.html)  
[Reference 2](https://en.wikipedia.org/wiki/Support_vector_machine)  
[Reference 3](http://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf)

The SVM classification problem (directly taken from sklearn website):  

Given training vectors $x_i\in \mathbb{R}^P$, $i=1,...,n$, and a vector $y\in\{1,-1\}^n$  
The goal is to find $w\in \mathbb{R}^P$ and $b\in \mathbb{R}$ such that prediction given by $\text{sign}(w^T\phi(x)+b)$ is accurate for most samples  

Solve the following __primal problem__:  
$$\begin{align}
\min_{w,b,\zeta}\frac{1}{2}w^Tw+C\sum^n_{i=1}\zeta_i\\
\text{subject to }y_i(w^T\phi(x_i)+b)\geq1-\zeta_i,\\
\zeta_i\geq0,i=1,...,n
\end{align}$$

Intuition is we are maximizing the margin (by minimizing $\|w\|^2=w^Tw$), while penalize when a sample is misclassified

The __dual problem__ to primal is: 
$$
\begin{align}
\min_\alpha\frac{1}{2}\alpha^TQ\alpha-e^T\alpha\\
\text{subject to }y^T\alpha=0,\\
0\leq\alpha_i\leq C,i=1,...,n
\end{align}
$$

$e$ is vector of all ones, $Q$ is nxn positive semidefinite matrix, $Q_{ij}\equiv y_iy_jK(x_i,x_j)$, and $K(x_i,x_j)=\phi(x_i)^T\phi(x_j)$ is the kernel  
$\alpha_i$ are dual coefficients, upper-bounded by $C$  
Highlighting the fact that training vectors are implicitly mapped to a higher (or infinite) dimensional space by function $\phi$  

After optimization problem solved, the __output__ for a given sample $x$ is:
$$\sum_{i\in SV}y_i\alpha_iK(x_i,x)+b$$


The problem solved by `liblinear` for `LinearSVC` is a equivant form of primal problem:
$$\min_{w,b}\frac{1}{2}w^Tw+C\sum_{i=1}\max(0,y_i(w^T\phi(x_i)+b))$$
which does not involve inner products between samples, and therefore cannot apply kernel tricks  

$$C=\frac{1}{\text{alpha}}$$

In [1]:
import Random
import Statistics
import LinearAlgebra

In [2]:
include("../tools.jl")
import .JuTools

### Linear SVM Implementation (Hinge Loss)  
[Reference](https://stackoverflow.com/questions/48804198/soft-margin-in-linear-support-vector-machine-using-python)  

Cost function is:
$$J=\frac{1}{2}w^Tw+\frac{C}{N}\sum^N_{i=1}\max\Big(0,1-y_i(w^T\phi(x_i)+b)\Big)$$

Gradient function for $w$ is:
$$\frac{\partial J}{\partial w}=w + \frac{C}{N}\sum^N_{i=1}\begin{cases}
0 & y_i(w^T\phi(x_i)+b) \geq 1\\
-y_i\phi(x_i) & \text{otherwise}
\end{cases}$$

Gradient function for $b$ is:
$$\frac{\partial J}{\partial b}=\frac{C}{N}\sum^N_{i=1}\begin{cases}
0 & y_i(w^T\phi(x_i)+b) \geq 1\\
-y_i & \text{otherwise}
\end{cases}$$

In [3]:
# define a struct to store weights
# this should be returned by a training function
# alpha should be treated as constant
mutable struct WeightsLinearSVM
    C::AbstractFloat
    w::Array{T} where T<:AbstractFloat
    b::AbstractFloat
end

In [4]:
# define cost function for linear SVM
# assum Y_data is {-1, 1}
function cost(X_data::Array{T} where T<:Number, Y_data::Array{T} where T<:Number, weights::WeightsLinearSVM)::AbstractFloat
    @assert ndims(Y_data) == ndims(weights.w) == 1
    @assert size(X_data) == (size(Y_data)[1], size(weights.w)[1])
    loss_w = 0.5 * (weights.w' * weights.w)
    loss_inner = 1.0 .- Y_data .* vec(X_data * weights.w .+ weights.b)
    loss_inner .= map(m->max(0.0,m), loss_inner)
    loss = loss_w + weights.C * sum(loss_inner) / size(X_data)[1]
    return loss
end

cost (generic function with 1 method)

In [5]:
X_data, Y_data = JuTools.data_generate_linear_2d()
Y_data .= Y_data .* 2.0 .- 1.0 # convert from {0,1} to {-1,1}
X_train, X_test, Y_train, Y_test = JuTools.split_data(X_data, Y_data)
println(size(X_train))
println(size(X_test))
println(size(Y_train))
println(size(Y_test))

(700, 2)
(300, 2)
(700,)
(300,)


In [6]:
weight_test = WeightsLinearSVM(1.0, Random.randn(size(X_data)[2]), Random.randn())
cost(X_data, Y_data, weight_test)

5.320976290385327

In [7]:
# define the learning function (gradient descent)
function learn!(X_data::Array{T} where T<:Number, Y_data::Array{T} where T<:Number, weights::WeightsLinearSVM, alpha::AbstractFloat)
    @assert ndims(Y_data) == ndims(weights.w) == 1
    @assert size(X_data) == (size(Y_data)[1], size(weights.w)[1])
    # compute deciding feature
    decide = (Y_data .* (X_data * weights.w .+ weights.b)) .< 1 # (? < 1) will be 1, otherwise 0
    # update w
    gradient_w = weights.w .+ (weights.C / size(X_data)[1]) .* vec(-(Y_data .* decide)' * X_data)
    gradient_w .= gradient_w .* alpha
    weights.w .= weights.w .- gradient_w
    # update b
    gradient_b = (weights.C / size(X_data)[1]) * sum(-(Y_data .* decide))
    gradient_b *= alpha
    weights.b = weights.b - gradient_b
    return nothing
end

learn! (generic function with 1 method)

In [8]:
# define prediction function
function predict_proba(X_predict::Array{T} where T<:Number, weights::WeightsLinearSVM)::Array
    @assert ndims(X_predict) == 2
    @assert size(X_predict)[2] == size(weights.w)[1]
    prediction = vec(X_predict * weights.w .+ weights.b)
    return prediction
end

# output prediction is in {-1, 1}
function predict(X_predict::Array{T} where T<:Number, weights::WeightsLinearSVM)::Array
    @assert ndims(X_predict) == 2
    @assert size(X_predict)[2] == size(weights.w)[1]
    prediction = vec(X_predict * weights.w .+ weights.b)
    prediction .= map(m -> m >= 0 ? 1.0 : -1.0, prediction)
    return prediction
end

predict (generic function with 1 method)

In [9]:
# training function for linear SVM
# assume Y_data is in {-1, 1}
# this function is similar to the training function for Logistic Regression (Both are gradient descent)
function train_linear(X_data::Array{T} where T<:Number, Y_data::Array{T} where T<:Number, C::AbstractFloat;
        learning_rate::AbstractFloat=0.1, max_iter::Integer=1000, n_iter_no_change::Integer=5, tol::AbstractFloat=0.001,
        verbose::Bool=false, shuffle::Bool=true, early_stop::Bool=true)::WeightsLinearSVM
    @assert ndims(X_data) == ndims(Y_data) + 1 == 2
    @assert size(X_data)[1] == size(Y_data)[1]
    @assert max_iter >= 0
    @assert n_iter_no_change >= 0
    @assert tol >= 0
    X_data = Float64.(X_data)
    Y_data = Float64.(Y_data)
    if shuffle
        JuTools.shuffle_data!(X_data, Y_data)
    end
    # is it better to use zero weights than normal weights ?
    weights = WeightsLinearSVM(C, Random.randn(size(X_data)[2]), Random.randn())
    best_cost = nothing
    n_cost_no_change = n_iter_no_change
    for i in 1:max_iter
        if n_cost_no_change <= 0 && early_stop
            break
        end
        learn!(X_data, Y_data, weights, learning_rate)
        new_cost = cost(X_data, Y_data, weights)
        if verbose
            acc = JuTools.compute_accuracy(predict(X_data, weights), Y_data)
            println("Iter: $i")
            println("Cost = $new_cost")
            println("Accuracy = $acc")
            println()
        end
        if early_stop
            if best_cost === nothing || isnan(best_cost)
                best_cost = new_cost
            else
                if new_cost > best_cost - tol
                    n_cost_no_change -= 1
                else
                    best_cost = min(new_cost, best_cost)
                    n_cost_no_change = n_iter_no_change
                end
            end
        end
    end
    return weights
end

train_linear (generic function with 1 method)

In [10]:
weights = train_linear(X_train, Y_train, 1.0, learning_rate=0.005, max_iter=20, tol=0.001, verbose=true)

Iter: 1
Cost = 17.49437622239612
Accuracy = 0.7328571428571429

Iter: 2
Cost = 15.759469404970652
Accuracy = 0.7328571428571429

Iter: 3
Cost = 14.041864723704366
Accuracy = 0.7328571428571429

Iter: 4
Cost = 12.34138958978907
Accuracy = 0.7328571428571429

Iter: 5
Cost = 10.657873135989934
Accuracy = 0.7328571428571429

Iter: 6
Cost = 8.991146199472801
Accuracy = 0.7328571428571429

Iter: 7
Cost = 7.341041304802785
Accuracy = 0.7328571428571429

Iter: 8
Cost = 5.707392647112458
Accuracy = 0.7328571428571429

Iter: 9
Cost = 4.090036075437951
Accuracy = 0.7328571428571429

Iter: 10
Cost = 2.5478231645134954
Accuracy = 0.7757142857142857

Iter: 11
Cost = 1.4517810681302135
Accuracy = 0.8285714285714286

Iter: 12
Cost = 0.8625868525842104
Accuracy = 0.8785714285714286

Iter: 13
Cost = 0.5845534725127407
Accuracy = 0.9071428571428571

Iter: 14
Cost = 0.49697254099419136
Accuracy = 0.9171428571428571

Iter: 15
Cost = 0.4724237420776445
Accuracy = 0.92

Iter: 16
Cost = 0.4669039302737097
Acc

WeightsLinearSVM(1.0, [0.4534683639714476, -0.24542848197815037], 0.5737279523597234)

In [11]:
JuTools.compute_accuracy(predict(X_test, weights), Y_test)

0.9233333333333333